#Series Multivariantes: Mercado del oro y la plata

###Pasos para crear un modelo VAR con series temporales multivariantes:

1. Análisis exploratorios de los datos.
2. Dividir la serie en conjuntos de entrenamiento y prueba.
3. Prueba de estacionariedad.
4. Transformar la serie de entrenamiento si es necesario.
5. Construir un modelo VAR sobre las series transformadas.
6. Causalidad de Granger.
7. Diagnóstico del modelo.
8. Realizar pronósticos utilizando el modelo finalmente elegido.
9. Transformación inversa del pronóstico a la escala original.
10. Realizar una evaluación del pronóstico.

# Los datos: Yahoo Finance
Si quieres tener los datos de precios, entidades, activos, etc, actualizados a día de hoy, lo puedes hacer de la siguiente manera. Estos datos han sido obtenidos mediante [Yahoo Finance](https://es.finance.yahoo.com/), una plataforma donde puedes ver, obtener, estudiar, analizar y comparar los precios de cierre de diferentes de acciones de los mercados financieros. La forma de obtener los datos de Yahoo Finance en Python es mediante el paquete `yfinance` que tendremos que instalar e importar.

In [ ]:
!pip install yfinance
import yfinance

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from statsmodels.tools.eval_measures import rmse

## Precios del Oro, Plata, Petróleo, Índice SP500, Cambio USD/EUR y el Índice de rendimiento de bonos del tesoro a 10 años.

In [ ]:
raw_data = yfinance.download(tickers = "GC=F, SI=F, EUR=X, CL=F, ^GSPC, ^TNX",
                             start = "2005-01-07",
                              end = "2020-06-03", interval = "1d",
                             group_by = 'ticker',
                             auto_adjust = True)
raw_data

Únicamente queremos quedarnos con los datos de cierre

In [ ]:
df_comp=raw_data.copy()
df_comp['Gold'] = df_comp['GC=F'].Close[:]
#Seleccionamos la variable GC=F y, dentro de ella, la variable Close
df_comp['Silver'] = df_comp['SI=F'].Close[:]
df_comp['Oil'] = df_comp['CL=F'].Close[:]
df_comp['Treasury'] = df_comp['^TNX'].Close[:]
df_comp['SP500'] = df_comp['^GSPC'].Close[:]
df_comp['USD/EUR'] = df_comp['EUR=X'].Close[:]


In [ ]:
df_comp.head()

Eliminamos las variables que no vamos a utilizar:

In [ ]:
del df_comp['GC=F']
del df_comp['SI=F']
del df_comp['CL=F']
del df_comp['^TNX']
del df_comp['^GSPC']
del df_comp['EUR=X']

In [ ]:
df_comp.head()

In [ ]:
df_comp.isnull().sum()

In [ ]:
df_comp=df_comp.asfreq('b')
df_comp.isnull().sum()

In [ ]:
#Rellenamos valores faltantes
df_comp=df_comp.ffill()
df_comp.isnull().sum()

In [ ]:
df_comp.columns = ['Gold', 'Silver', 'Oil', 'Treasury','SP500','USD/EUR']
df_comp.head()

In [ ]:
df_comp.info()

In [ ]:
#Guardar el dataset
df_comp.to_csv('Market2020.csv')

# Análisis exploratorio

In [ ]:
# Plots
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=3, ncols=2, dpi=120, figsize=(8,6))
for i, ax in enumerate(axes.flatten()):
 data = df_comp[df_comp.columns[i]]
 ax.plot(data, color='red', linewidth=1)
 ax.set_title(df_comp.columns[i])
 ax.xaxis.set_ticks_position('none')
 ax.yaxis.set_ticks_position('none')
 ax.spines['top'].set_alpha(0)
 ax.tick_params(labelsize=6)
 plt.tight_layout();

In [ ]:
corr=df_comp.corr()
corr

In [ ]:
import seaborn as sns
sns.heatmap(corr, xticklabels=
            corr.columns.values,
            yticklabels=corr.columns.values,
            annot=True,vmax=1, vmin=-1,
            cmap =sns.diverging_palette(220, 10, as_cmap=True),
            center=0 )
plt.show()

# Dividir los datos en conjunto de Entrenamiento y Prueba
El modelo VAR se ajustará al conjunto de entrenamiento X_train y luego se utilizará el modelo para pronosticar las próximas 15 observaciones. Estos pronósticos se compararán con los datos reales del conjunto de prueba.

In [ ]:
n_obs=15
X_train, X_test = df_comp[0:-n_obs], df_comp[-n_obs:]
print(X_train.shape, X_test.shape)

# Prueba de estacionariedad
Necesitamos que los datos sean estacionarios para poder usar el modelo VAR. Vamos a averiguarlo haciendo un Test de Dickey-Fuller aumentado, a cada variable univariante que tiene nuestro dataset.

In [ ]:
import statsmodels.tsa.stattools as sts

In [ ]:
def augmented_dickey_fuller_statistics(time_series):
  result = sts.adfuller(time_series.values)
  print('p-value: %f' % result[1])

print('Test de Dickey-Fuller Aumentado:')
print('Serie de tiempo Precio del Oro')
augmented_dickey_fuller_statistics(X_train['Gold'])
print('Serie de tiempo Precio de la Plata')
augmented_dickey_fuller_statistics(X_train['Silver'])
print('Serie de tiempo Precio del Petróleo')
augmented_dickey_fuller_statistics(X_train['Oil'])
print('Serie de tiempo Índice del rendimiento de bonos del tesoro en 10 años')
augmented_dickey_fuller_statistics(X_train['Treasury'])
print('Serie de tiempo Índice SP500')
augmented_dickey_fuller_statistics(X_train['SP500'])
print('Serie de tiempo Cambio USD/EUR')
augmented_dickey_fuller_statistics(X_train['USD/EUR'])

Todos los p-valores son mayores que 0.05, con lo cual no se rechaza la hipótesis nula de que la serie no es estacionaria. Habría que transformar los datos porque no se puede confirmar estacionariedad.

# Transformación de los datos
La aplicación de la primera diferenciación en el conjunto de entrenamiento debería hacer que todas las series 6 sean estacionarias. Sin embargo, este es un proceso iterativo en el que, después de la primera diferenciación, es posible que la serie siga siendo no estacionaria. Tendremos que aplicar una segunda diferencia o transformación logarítmica para estandarizar la serie en tales casos.

In [ ]:
X_train_transformed=X_train.diff().dropna()
X_train_transformed.head()

In [ ]:
# Dibujemos los datos transformados
fig, axes = plt.subplots(nrows=3, ncols=2, dpi=120, figsize=(8,6))
for i, ax in enumerate(axes.flatten()):
  d = X_train_transformed[X_train_transformed.columns[i]]
  ax.plot(d, color='red', linewidth=1)
  ax.set_title(df_comp.columns[i])
  ax.xaxis.set_ticks_position('none')
  ax.yaxis.set_ticks_position('none')
  ax.spines['top'].set_alpha(0)
  ax.tick_params(labelsize=6)
  plt.tight_layout();

Chequeando si los datos transformados (diferenciados) son estacionarios:

In [ ]:
print('Test de Dickey-Fuller Aumentado:')
print('Serie de tiempo Precio del Oro Diferenciada')
augmented_dickey_fuller_statistics(X_train_transformed['Gold'])
print('Serie de tiempo Precio de la Plata Diferenciada')
augmented_dickey_fuller_statistics(X_train_transformed['Silver'])
print('Serie de tiempo Precio del Petróleo Diferenciada')
augmented_dickey_fuller_statistics(X_train_transformed['Oil'])
print('Serie de tiempo Índice del rendimiento de bonos del tesoro en 10 años Diferenciada')
augmented_dickey_fuller_statistics(X_train_transformed['Treasury'])
print('Serie de tiempo Índice SP500 Diferenciada')
augmented_dickey_fuller_statistics(X_train_transformed['SP500'])
print('Serie de tiempo Cambio USD/EUR Diferenciada')
augmented_dickey_fuller_statistics(X_train_transformed['USD/EUR'])

Todos los p-valores son < 0.05 por lo tanto se podría concluir que con un solo orden de diferenciación se obtienen series estacionarias.

# Modelo VAR

El modelo VAR (Vector AutoRegressive) es un modelo estadístico utilizado para analizar la relación entre múltiples series de tiempo. Es especialmente útil cuando se trata de modelar y comprender las interacciones entre variables económicas o financieras que evolucionan juntas en el tiempo.

VAR requiere la estacionariedad de la serie, lo que significa que la media de la serie no cambia con el tiempo (podemos verlo en el gráfico dibujado junto a la Prueba de Dickey-Fuller aumentada).

In [ ]:
from statsmodels.tsa.api import VAR

####Ejemplo básico de VAR

In [ ]:
from statsmodels.tools.eval_measures import rmse

# Crear una serie de tiempo sintética (reemplaza esto con tus datos)
np.random.seed(0)
data = np.random.randn(100, 2)
df = pd.DataFrame(data, columns=['Variable1', 'Variable2'])

# Especificar el modelo VAR con un retardo de 2
model = VAR(df)
results = model.fit(2)

# Resumen de la estimación del modelo
print(results.summary())

# Generar pronósticos para los próximos 5 pasos de tiempo
lag_order = results.k_ar
forecast = results.forecast(df.values[-lag_order:], steps=5)

# Visualizar los pronósticos
forecast_df = pd.DataFrame(forecast, columns=['Variable1_Forecast', 'Variable2_Forecast'])
print(forecast_df)


In [ ]:
# Calcular el error RMSE (Root Mean Squared Error) de los pronósticos
actual_data = df.iloc[-5:]
rmse_score = rmse(actual_data.values, forecast_df.values)
print(f'RMSE Variable1: {rmse_score[0]:.2f}')
print(f'RMSE Variable2: {rmse_score[1]:.2f}')

## Seleccionando el orden del modelo

In [ ]:
# Modelo para el problema que nos ocupa:
model = VAR(X_train_transformed)

In [ ]:
modelsel=model.select_order(15)
modelsel.summary()

## Ajustando el modelo

In [ ]:
res = model.fit(maxlags=15, ic='aic')
res.summary()

# Causalidad de Granger

Si el p-valor < 0.05 Rechazo la hipótesis nula H0 y mantengo la variable dentro del modelo.

### Gold

In [ ]:
grangercaus=res.test_causality(['Silver', 'Oil', 'Treasury','SP500','USD/EUR'],
                               ['Gold'],kind='f')
grangercaus.summary()

### Silver

In [ ]:
grangercaus=res.test_causality(['Gold', 'Oil', 'Treasury','SP500','USD/EUR'],
                               ['Silver'],kind='f')
grangercaus.summary()

### Oil

In [ ]:
grangercaus=res.test_causality(['Gold','Silver','Treasury','SP500','USD/EUR'],
                               ['Oil'],kind='f')
grangercaus.summary()

### Treasury

In [ ]:
grangercaus=res.test_causality(['Gold','Silver','Oil','SP500','USD/EUR'],
                               ['Treasury'],kind='f')
grangercaus.summary()

### SP500

In [ ]:
grangercaus=res.test_causality(['Gold','Silver','Oil','Treasury','USD/EUR'],
                               ['SP500'],kind='f')
grangercaus.summary()

### USD/EUR

In [ ]:
grangercaus=res.test_causality(['Gold','Silver','Oil','Treasury','SP500'],
                               ['USD/EUR'],kind='f')
grangercaus.summary()

# Matriz de causalidad de Granger

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=15
test = 'ssr_chi2test'
def grangers_causality_matrix(X_train_transformed, variables, test = 'ssr_chi2test', verbose=False):
  dataset = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
  for c in dataset.columns:
    for r in dataset.index:
      test_result = grangercausalitytests(X_train_transformed[[r,c]], maxlag=maxlag, verbose=False)
      p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
      if verbose:
        print(f'Y = {r}, X = {c}, P Values = {p_values}')
      min_p_value = np.min(p_values)
      dataset.loc[r,c] = min_p_value
  dataset.columns = [var + '_x' for var in variables]
  dataset.index = [var + '_y' for var in variables]
  return dataset
grangers_causality_matrix(X_train_transformed, variables = X_train_transformed.columns)

# Diagnosis del modelo

## Residuos

In [ ]:
residuals=res.resid

In [ ]:
fig, axs = plt.subplots(6)
fig.suptitle('Gráficos de los residuos',fontsize=20)
fig.set_size_inches(18, 10)
[axs[i].plot(residuals.iloc[:,i]) for i in range(6)]
plt.show()

## Dickey - Fuller a los residuos

In [ ]:
print('Gold Silver Oil Treas. SP500 USD/EUR')
[sts.adfuller(residuals.iloc[:,i])[1] for i in range(6)]


Todos los p-valores son < 0.05 por tanto se rechaza la hipótesis nula de que las 6 series de residuos no son estacionarias, por lo cual con un 95% de confianza se cree que son estacionarias.

## ACF de los Residuos

In [ ]:
import statsmodels.graphics.tsaplots as sgt
[sgt.plot_acf(residuals.iloc[:,i], zero = False, lags = 40) for i in range(6)]
plt.show()


Conclusión: Los residuos del modelo no presentan estructura de autocorrelación, son estacionarios según los resultados de la prueba de Dickey - Fuller aumentada y en los gráficos se puede comprobar esto visualmente, entonces puede concluirse que son ruido blanco como es deseable.

## Valores predichos

In [ ]:
y_fitted = res.fittedvalues
fig, axs = plt.subplots(6)
fig.suptitle('Gráficos de los valores predichos por el modelo',fontsize=20)
fig.set_size_inches(18, 10)
[axs[i].plot(y_fitted.iloc[:,i]) for i in range(6)]
plt.show()

# Pronósticos a futuro

## Hallando los pronósticos

Para pronosticar, al modelo VAR le podemos pasar hasta el número de orden de retraso de observaciones de los datos pasados. Esto se debe a que los términos en el modelo VAR son esencialmente los retrasos de las diversas series de tiempo en el conjunto de datos, por lo que debemos proporcionar tantos valores anteriores como lo indique el orden de retraso utilizado por el modelo. De lo contrario estaríamos introduciendo poca fiabilidad.


In [ ]:
# Obtener el orden del modelo
lag_order = res.k_ar
print('Orden del modelo:', lag_order)
# Input data para hacer forecasting (pronósticos a futuro)
input_data = X_train_transformed.values[-lag_order:]
# Forecasting
pred = res.forecast(y=input_data, steps=n_obs)
pred = (pd.DataFrame(pred, index=X_test.index, columns=X_test.columns + '_pred'))
print('Predicciones:')
pred

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (12, 10))
res.plot_forecast(15)
plt.tight_layout(h_pad = 1.15)
plt.show()

## Invirtiendo la transformación de los pronósticos a la escala original

Los pronósticos se generan en la escala de los datos de entrenamiento utilizados por el modelo, es decir, son datos transformados. Entonces, para volver a ponerlos en su escala original, necesitamos des-diferenciarlos.
La forma de invertir la diferenciación es sumar estas diferencias consecutivamente al número base. Una forma sencilla de hacerlo es determinar primero la suma acumulada y luego sumarla al número base.
Este proceso se puede revertir agregando la observación en el paso de tiempo anterior al difference value. inverted(ts) = differenced(ts) + observation(ts-1)

In [ ]:
# Invirtiendo la transformación
def invert_transformation(X_train, pred):
  forecast = pred.copy()
  columns = X_train.columns
  for col in columns:
    forecast[str(col)+'_pred'] = X_train[col].iloc[-1] + forecast[str(col)+'_pred'].cumsum()
  return forecast

output = invert_transformation(X_train, pred)
output


## Pronósticos del Oro

In [ ]:
plt.figure(figsize = (9,7))
plt.plot(output.iloc[:,0])
plt.title('Gold Forecast')
plt.grid()
plt.show()

## Comparando los datos pronosticados con el dataset de prueba

In [ ]:
combine = pd.concat([output['Gold_pred'], X_test['Gold']], axis=1)
combine = combine.round(decimals=2)
combine = combine.reset_index()
combine = combine.sort_values(by='Date', ascending=False)

In [ ]:
combine

In [ ]:
plt.figure(figsize=(8,5))
#plt.plot(X_train['Gold'],label="Train")
plt.plot(X_test['Gold'],label="Test")
plt.plot(output['Gold_pred'],label="Prediccion")
plt.legend()
#plt.ylim(0, 1800)
plt.show()

# Evaluación del modelo

Para evaluar los pronósticos, se puede calcular un conjunto completo de métricas, como MAE y RMSE.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
print('Mean absolute error:', mean_absolute_error(
    combine['Gold'].values, combine['Gold_pred'].values))
print('Root mean squared error:', np.sqrt(
    mean_squared_error(combine['Gold'].values, combine['Gold_pred'].values)))